In [2]:
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.chat_models import ChatOllama

vectorstore = FAISS.from_texts(
    ["harrison worked at kensho", "bears like to eat honey"],
    embedding=OllamaEmbeddings(model="mistral"),
)

retriever = vectorstore.as_retriever()

prompt = ChatPromptTemplate.from_messages([
  ("system", "Answer the question based only on the following context: {context}"),
  ("human", "Question: {question}")
])

model = ChatOllama(model="mistral")

retrieval = RunnableParallel(context=retriever, question=RunnablePassthrough())

chain = retrieval | prompt | model | StrOutputParser()

chain.invoke("where did harrison work?")

" Harrison worked at Kensho, based on the given context, there is no relation between the context about bears and honey with Harrison's place of work."

# Using `itemgetter` as shorthand

In [4]:
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
  ("system", "Answer the question based only on the following context: {context}"),
  ("system", "Answer in the following language: {language}"),
  ("human", "Question: {question}")
])

retrieval = RunnableParallel(
  context=itemgetter("question") | retriever,
  question=itemgetter("question") | RunnablePassthrough(), 
  language=itemgetter("language") | RunnablePassthrough()
)

chain = retrieval | prompt | model | StrOutputParser()

chain.invoke({"question": "where did harrison work", "language": "italian"})

' Risposta in lingua italiana: Harrison ha lavorato a Kensho. (Harrison worked at Kensho.)'